Advanced RAG: Contextual RAG
========================================

Welcome to the Gravix Layer Advanced Cookbook! This notebook offers a comprehensive, hands-on guide to building a sophisticated **Contextual Retrieval-Augmented Generation (RAG)** system using Gravix Layer's powerful Inference APIs and Vector Database Service. Designed for seamless integration of conversational context and dynamic retrieval, this system elevates traditional RAG by incorporating advanced contextual understanding and query enhancement.

What You'll Learn:
------------------
-   How to configure Gravix Layer APIs for semantic search and context-aware text generation
-   How to ingest and process diverse document formats to build a robust knowledge base
-   How to implement contextual retrieval with query expansion and reranking for precise results
-   How to leverage conversation history for personalized, coherent responses
-   How to manage and extend your RAG pipeline with advanced features like metadata filtering and conversation export

Who Is This For?
----------------

-   AI developers and data scientists aiming to build cutting-edge RAG systems with contextual intelligence
-   Researchers and engineers exploring Gravix Layer's SDK for interactive, real-world applications
-   Enthusiasts interested in conversational AI and advanced retrieval techniques

Architecture Overview
---------------------

1.  **Document Ingestion & Chunking**: Process multi-format documents (PDF, DOCX, TXT, Markdown) into contextually aware chunks with rich metadata
2.  **Vector Storage & Embedding**: Convert text into embeddings using Gravix Layer's vector API and store them in a scalable vector database
3.  **Contextual Retrieval**: Perform semantic search with dynamic query expansion and reranking, leveraging conversation history for relevance
4.  **Context-Aware Generation**: Combine retrieved context and conversational memory with Gravix Layer's LLM for accurate, coherent responses
5.  **Memory & Interaction Management**: Maintain persistent conversation history and support real-time document uploads and interactive querying

This implementation goes beyond traditional RAG by integrating conversation-aware retrieval, multi-stage search, and personalized response generation, making it ideal for interactive applications like chatbots, knowledge assistants, and research tools. Let's dive in and unlock the full potential of Contextual RAG with Gravix Layer!

In [35]:
# Install required packages
!pip install gravixlayer PyPDF2 python-docx requests python-dotenv -q
print("✅ All packages installed successfully!")

✅ All packages installed successfully!


In [58]:
import os
import tempfile
import time
import json
import re
import uuid
from typing import List, Optional, Tuple, Any, Dict
from datetime import datetime
from collections import defaultdict, deque

import requests
import PyPDF2
import docx
from gravixlayer import GravixLayer
from dotenv import load_dotenv
from IPython.display import display, Markdown, HTML

In [ ]:
# Load environment variables
load_dotenv()

# Configuration
GRAVIX_API_KEY = os.getenv('GRAVIXLAYER_API_KEY')
if not GRAVIX_API_KEY:
    print("⚠️  GRAVIXLAYER_API_KEY not found in environment variables")
    print("Please set it using: set_api_key('your_api_key_here')")

# Global configuration
CONFIG = {
    'base_url': 'https://api.gravixlayer.com/v1',
    'embedding_model': 'baai/bge-large-en-v1.5',
    'llm_model': 'meta-llama/llama-3.1-8b-instruct',
    'vector_dimension': 1024,
    'similarity_metric': 'cosine',
    'chunk_size': 800,
    'chunk_overlap': 150,
    'max_conversation_history': 10,
    'retrieval_k': 5,
    'rerank_k': 3
}

print("📋 Contextual RAG Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")
print("\n✅ Imports and configuration complete!")

📋 Contextual RAG Configuration:
  base_url: https://api.gravixlayer.com/v1
  embedding_model: baai/bge-large-en-v1.5
  llm_model: meta-llama/llama-3.1-8b-instruct
  vector_dimension: 1024
  similarity_metric: cosine
  chunk_size: 800
  chunk_overlap: 150
  max_conversation_history: 10
  retrieval_k: 5
  rerank_k: 3

✅ Imports and configuration complete!


In [37]:
# Initialize global session state for notebook
def init_session_state() -> None:
    """Initialize session state variables for the notebook"""
    global session_state
    try:
        session_state
    except NameError:
        session_state = {
            "api_key_submitted": False,
            "gravix_api_key": GRAVIX_API_KEY or "",
            "index_id": "",
            "chat_history": deque(maxlen=CONFIG['max_conversation_history']),
            "processed_files": [],
            "document_chunks": {},
            "conversation_context": {},
            "last_query": "",
            "last_response": "",
            "retrieval_metadata": {},
            "total_chunks_stored": 0
        }

# Helper functions for API key management
def set_api_key(api_key: str) -> None:
    """Set the GravixLayer API key"""
    init_session_state()
    session_state['gravix_api_key'] = api_key
    session_state['api_key_submitted'] = True
    os.environ['GRAVIXLAYER_API_KEY'] = api_key
    print(f"✅ API key set successfully")

def verify_api_key(api_key: str = None) -> bool:
    """Verify the GravixLayer API key"""
    init_session_state()
    key = api_key or session_state.get('gravix_api_key') or GRAVIX_API_KEY
    if not key:
        print("❌ No API key provided")
        return False
    
    try:
        headers = {"Authorization": f"Bearer {key}"}
        response = requests.get(f"{CONFIG['base_url']}/vectors/indexes", headers=headers)
        if response.status_code == 200:
            set_api_key(key)
            print("✅ API key verified successfully!")
            return True
        else:
            print(f"❌ API key verification failed: {response.text}")
            return False
    except Exception as e:
        print(f"❌ Error verifying API key: {e}")
        return False

def ensure_client():
    """Ensure GravixLayer client is properly initialized"""
    init_session_state()
    api_key = session_state.get("gravix_api_key") or GRAVIX_API_KEY
    if not api_key:
        raise ValueError("GravixLayer API key not provided. Use set_api_key() or verify_api_key()")
    os.environ['GRAVIXLAYER_API_KEY'] = api_key
    return GravixLayer()

init_session_state()
print("📝 Session state initialized")
print(f"💬 Conversation history limit: {CONFIG['max_conversation_history']} interactions")

📝 Session state initialized
💬 Conversation history limit: 10 interactions


In [38]:
def create_vector_index(name: str, dimension: int = None) -> Optional[str]:
    """Create a new vector index using GravixLayer API"""
    init_session_state()
    api_key = session_state.get('gravix_api_key')
    if not api_key:
        print("❌ No API key available. Use verify_api_key() first.")
        return None
    
    dimension = dimension or CONFIG['vector_dimension']
    
    try:
        response = requests.post(
            f"{CONFIG['base_url']}/vectors/indexes",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json",
            },
            json={
                "name": name,
                "dimension": dimension,
                "metric": CONFIG['similarity_metric']
            }
        )
        
        if response.status_code in [200, 201]:
            index_data = response.json()
            index_id = index_data.get("id")
            session_state['index_id'] = index_id
            print(f"✅ Created vector index: {index_id}")
            return index_id
        else:
            print(f"❌ Failed to create index: {response.text}")
            return None
    except Exception as e:
        print(f"❌ Error creating vector index: {e}")
        return None

def list_vector_indexes() -> List[Dict]:
    """List all available vector indexes"""
    init_session_state()
    api_key = session_state.get('gravix_api_key')
    if not api_key:
        return []
    
    try:
        response = requests.get(
            f"{CONFIG['base_url']}/vectors/indexes",
            headers={"Authorization": f"Bearer {api_key}"}
        )
        if response.status_code == 200:
            return response.json().get("indexes", [])
        return []
    except Exception:
        return []

def find_or_create_index(name: str) -> Optional[str]:
    """Find existing index by name or create new one"""
    indexes = list_vector_indexes()
    for idx in indexes:
        if idx.get('name') == name:
            index_id = idx.get('id')
            session_state['index_id'] = index_id
            print(f"✅ Using existing index: {index_id}")
            return index_id
    
    return create_vector_index(name)

def upsert_vectors(text_chunks: List[str], metadata_list: List[Dict]) -> bool:
    """Upload text chunks to GravixLayer vector database"""
    init_session_state()
    api_key = session_state.get('gravix_api_key')
    index_id = session_state.get('index_id')
    
    if not api_key or not index_id:
        print("❌ Missing API key or index ID")
        return False
    
    try:
        vectors_data = {"vectors": []}
        
        for i, (chunk, metadata) in enumerate(zip(text_chunks, metadata_list)):
            chunk_id = f"{metadata.get('filename', 'doc')}_{i}_{uuid.uuid4().hex[:8]}"
            
            vector_entry = {
                "id": chunk_id,
                "text": chunk,
                "model": CONFIG['embedding_model'],
                "metadata": {
                    **metadata,
                    "chunk_id": chunk_id,
                    "chunk_index": i,
                    "timestamp": datetime.now().isoformat(),
                    "total_chunks": len(text_chunks)
                },
                "delete_protection": False
            }
            vectors_data["vectors"].append(vector_entry)
            
            # Store in session for retrieval
            session_state['document_chunks'][chunk_id] = chunk
        
        response = requests.post(
            f"{CONFIG['base_url']}/vectors/{index_id}/text/upsert",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            },
            json=vectors_data
        )
        
        if response.status_code in [200, 201]:
            result = response.json()
            upserted_count = result.get('upserted_count', len(text_chunks))
            session_state['total_chunks_stored'] += upserted_count
            print(f"✅ Uploaded {upserted_count} chunks successfully")
            return True
        else:
            print(f"❌ Failed to upload vectors: {response.text}")
            return False
            
    except Exception as e:
        print(f"❌ Error uploading vectors: {e}")
        return False

def search_vectors(query: str, top_k: int = None, filters: Dict = None) -> List[Dict]:
    """Search for similar text chunks in vector database"""
    init_session_state()
    api_key = session_state.get('gravix_api_key')
    index_id = session_state.get('index_id')
    top_k = top_k or CONFIG['retrieval_k']
    
    if not api_key or not index_id:
        print("❌ Missing API key or index ID")
        return []
    
    try:
        search_data = {
            "query": query,
            "model": CONFIG['embedding_model'],
            "top_k": top_k,
            "include_metadata": True,
            "include_values": False
        }
        
        if filters:
            search_data["filter"] = filters
        
        response = requests.post(
            f"{CONFIG['base_url']}/vectors/{index_id}/search/text",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            },
            json=search_data
        )
        
        if response.status_code == 200:
            result = response.json()
            hits = result.get("hits", [])
            print(f"🔍 Found {len(hits)} relevant chunks")
            return hits
        else:
            print(f"❌ Search failed: {response.text}")
            return []
            
    except Exception as e:
        print(f"❌ Error searching vectors: {e}")
        return []

print("✅ Vector operations functions ready")

✅ Vector operations functions ready


In [39]:
def extract_text_from_file(file_content: bytes, filename: str) -> str:
    """Extract text from various file formats with enhanced error handling"""
    try:
        file_ext = os.path.splitext(filename)[1].lower()
        
        if file_ext == '.pdf':
            return _extract_pdf_text(file_content)
        elif file_ext == '.docx':
            return _extract_docx_text(file_content)
        elif file_ext in ['.txt', '.md']:
            return file_content.decode('utf-8')
        else:
            # Try as plain text
            return file_content.decode('utf-8', errors='ignore')
            
    except Exception as e:
        print(f"❌ Failed to extract text from {filename}: {e}")
        return ""

def _extract_pdf_text(file_content: bytes) -> str:
    """Extract text from PDF bytes"""
    with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp:
        tmp.write(file_content)
        tmp_path = tmp.name
    
    try:
        text = ""
        with open(tmp_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page_num, page in enumerate(pdf_reader.pages, 1):
                try:
                    page_text = page.extract_text()
                    if page_text.strip():
                        text += f"\n\n[PAGE {page_num}]\n{page_text}"
                except Exception as e:
                    print(f"⚠️  Warning: Could not extract page {page_num}: {e}")
                    continue
        return text.strip()
    finally:
        os.unlink(tmp_path)

def _extract_docx_text(file_content: bytes) -> str:
    """Extract text from DOCX bytes"""
    with tempfile.NamedTemporaryFile(delete=False, suffix='.docx') as tmp:
        tmp.write(file_content)
        tmp_path = tmp.name
    
    try:
        doc = docx.Document(tmp_path)
        paragraphs = []
        for paragraph in doc.paragraphs:
            text = paragraph.text.strip()
            if text:
                paragraphs.append(text)
        return "\n\n".join(paragraphs)
    finally:
        os.unlink(tmp_path)

def chunk_text_contextually(text: str, filename: str, chunk_size: int = None, overlap: int = None) -> List[Dict]:
    """Split text into overlapping chunks with contextual metadata"""
    chunk_size = chunk_size or CONFIG['chunk_size']
    overlap = overlap or CONFIG['chunk_overlap']
    
    if len(text) <= chunk_size:
        return [{
            'text': text,
            'metadata': {
                'filename': filename,
                'chunk_number': 0,
                'char_start': 0,
                'char_end': len(text),
                'document_type': _get_document_type(filename)
            }
        }] if text.strip() else []
    
    # Enhanced chunking with sentence awareness
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks = []
    current_chunk = ""
    current_start = 0
    chunk_number = 0
    
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
        
        # Check if adding sentence exceeds chunk size
        if len(current_chunk) + len(sentence) + 1 > chunk_size and current_chunk:
            if len(current_chunk.strip()) >= 50:  # Minimum chunk size
                chunk_end = current_start + len(current_chunk)
                chunks.append({
                    'text': current_chunk.strip(),
                    'metadata': {
                        'filename': filename,
                        'chunk_number': chunk_number,
                        'char_start': current_start,
                        'char_end': chunk_end,
                        'document_type': _get_document_type(filename),
                        'has_previous': chunk_number > 0,
                        'context_summary': _generate_context_summary(current_chunk)
                    }
                })
                chunk_number += 1
            
            # Handle overlap for context continuity
            words = current_chunk.split()
            overlap_words = words[-overlap//10:] if len(words) > overlap//10 else []
            overlap_text = " ".join(overlap_words)
            
            current_chunk = overlap_text + " " + sentence if overlap_words else sentence
            current_start = chunk_end - len(overlap_text) if overlap_words else chunk_end
        else:
            current_chunk += (" " + sentence) if current_chunk else sentence
    
    # Add final chunk
    if current_chunk.strip() and len(current_chunk.strip()) >= 50:
        chunks.append({
            'text': current_chunk.strip(),
            'metadata': {
                'filename': filename,
                'chunk_number': chunk_number,
                'char_start': current_start,
                'char_end': current_start + len(current_chunk),
                'document_type': _get_document_type(filename),
                'has_previous': chunk_number > 0,
                'is_final': True,
                'context_summary': _generate_context_summary(current_chunk)
            }
        })
    
    return chunks

def _get_document_type(filename: str) -> str:
    """Determine document type from filename"""
    ext = os.path.splitext(filename)[1].lower()
    type_map = {
        '.pdf': 'pdf',
        '.docx': 'docx', 
        '.txt': 'text',
        '.md': 'markdown'
    }
    return type_map.get(ext, 'unknown')

def _generate_context_summary(text: str) -> str:
    """Generate a brief context summary for the chunk"""
    # Simple keyword extraction
    words = re.findall(r'\b\w{4,}\b', text.lower())
    word_freq = {}
    for word in words:
        word_freq[word] = word_freq.get(word, 0) + 1
    
    # Get top 5 keywords
    top_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:5]
    return ", ".join([word for word, _ in top_words])

print("✅ Text processing functions ready")

✅ Text processing functions ready


In [40]:
def add_to_conversation(role: str, content: str, metadata: Dict = None) -> None:
    """Add a message to the conversation history"""
    init_session_state()
    
    message = {
        "role": role,
        "content": content,
        "timestamp": datetime.now().isoformat(),
        "metadata": metadata or {}
    }
    
    session_state['chat_history'].append(message)
    
    if role == "user":
        session_state['last_query'] = content
    elif role == "assistant":
        session_state['last_response'] = content

def get_conversation_context(include_last_n: int = 3) -> str:
    """Get recent conversation context for contextual processing"""
    init_session_state()
    
    if not session_state['chat_history']:
        return ""
    
    recent_messages = list(session_state['chat_history'])[-include_last_n * 2:]  # Get Q&A pairs
    context_parts = []
    
    for msg in recent_messages:
        role_prefix = "Human" if msg['role'] == 'user' else "Assistant"
        content = msg['content'][:200] + "..." if len(msg['content']) > 200 else msg['content']
        context_parts.append(f"{role_prefix}: {content}")
    
    return "\n".join(context_parts)

def extract_conversation_keywords() -> List[str]:
    """Extract key terms from recent conversation for context"""
    init_session_state()
    
    if not session_state['chat_history']:
        return []
    
    # Get text from last 3 interactions
    recent_text = " ".join([
        msg['content'] for msg in list(session_state['chat_history'])[-6:]
    ])
    
    # Simple keyword extraction
    words = re.findall(r'\b\w{4,}\b', recent_text.lower())
    word_freq = {}
    for word in words:
        if word not in ['this', 'that', 'with', 'have', 'will', 'they', 'from', 'been', 'said']:
            word_freq[word] = word_freq.get(word, 0) + 1
    
    # Return top keywords
    top_keywords = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:8]
    return [word for word, count in top_keywords if count > 1]

def clear_conversation() -> None:
    """Clear conversation history"""
    init_session_state()
    session_state['chat_history'].clear()
    session_state['conversation_context'].clear()
    session_state['last_query'] = ""
    session_state['last_response'] = ""
    print("💭 Conversation history cleared")

def show_conversation_history() -> None:
    """Display the conversation history"""
    init_session_state()
    
    if not session_state['chat_history']:
        print("📝 No conversation history yet")
        return
    
    display(Markdown("### 💬 Conversation History"))
    
    for i, msg in enumerate(session_state['chat_history'], 1):
        role_emoji = "🧑" if msg['role'] == 'user' else "🤖"
        role_name = "You" if msg['role'] == 'user' else "Assistant"
        timestamp = datetime.fromisoformat(msg['timestamp']).strftime("%H:%M:%S")
        
        print(f"\n{role_emoji} **{role_name}** ({timestamp}):")
        print(f"{msg['content']}")
        print("─" * 60)

print("✅ Conversation management functions ready")

✅ Conversation management functions ready


In [41]:
def contextual_search_and_rerank(query: str, conversation_context: str = "", 
                                top_k: int = None) -> Tuple[List[Dict], str]:
    """Perform contextual search with query expansion and reranking"""
    top_k = top_k or CONFIG['retrieval_k']
    
    print(f"🔍 Starting contextual search for: '{query[:50]}...'")
    
    # Step 1: Generate expanded queries
    expanded_queries = _expand_query_contextually(query, conversation_context)
    print(f"📝 Generated {len(expanded_queries)} search variations")
    
    # Step 2: Search with multiple queries
    all_hits = []
    for expanded_query in expanded_queries:
        hits = search_vectors(expanded_query, top_k=top_k * 2)  # Get more for reranking
        all_hits.extend(hits)
    
    # Step 3: Deduplicate and merge scores
    unique_hits = _deduplicate_hits(all_hits)
    print(f"🎯 Found {len(unique_hits)} unique results")
    
    # Step 4: Contextual reranking
    reranked_hits = _rerank_by_context(unique_hits, query, conversation_context)
    final_hits = reranked_hits[:CONFIG['rerank_k']]
    
    print(f"⭐ Selected top {len(final_hits)} most relevant chunks")
    
    # Step 5: Build context string
    context_string = _build_context_string(final_hits, query, conversation_context)
    
    return final_hits, context_string

def _expand_query_contextually(query: str, conversation_context: str) -> List[str]:
    """Expand query based on conversation context"""
    expanded_queries = [query]  # Original query
    
    # Add conversation keywords if available
    conversation_keywords = extract_conversation_keywords()
    if conversation_keywords:
        # Create keyword-enhanced query
        keyword_query = f"{query} {' '.join(conversation_keywords[:3])}"
        expanded_queries.append(keyword_query)
    
    # Create semantic variations
    query_words = query.lower().split()
    if len(query_words) > 2:
        # Partial query with key terms
        partial_query = ' '.join(query_words[:2] + query_words[-1:])
        expanded_queries.append(partial_query)
    
    # Add context-specific variations based on question type
    if query.lower().startswith(('what', 'how', 'why', 'when', 'where')):
        # Question-specific expansion
        base_terms = ' '.join(query_words[1:])  # Remove question word
        expanded_queries.append(base_terms)
    
    return list(set(expanded_queries))  # Remove duplicates

def _deduplicate_hits(hits: List[Dict]) -> List[Dict]:
    """Remove duplicate hits and merge scores"""
    hit_dict = {}
    
    for hit in hits:
        hit_id = hit.get('id')
        if hit_id in hit_dict:
            # Keep hit with higher score
            existing_score = hit_dict[hit_id].get('score', 0)
            new_score = hit.get('score', 0)
            if new_score > existing_score:
                hit_dict[hit_id] = hit
        else:
            hit_dict[hit_id] = hit
    
    # Sort by score descending
    return sorted(hit_dict.values(), key=lambda x: x.get('score', 0), reverse=True)

def _rerank_by_context(hits: List[Dict], query: str, conversation_context: str) -> List[Dict]:
    """Rerank hits based on contextual relevance"""
    conversation_keywords = extract_conversation_keywords()
    
    for hit in hits:
        base_score = hit.get('score', 0)
        context_bonus = 0
        
        # Get chunk text for analysis
        chunk_id = hit.get('id')
        chunk_text = session_state['document_chunks'].get(chunk_id, '').lower()
        
        if chunk_text:
            # Bonus for conversation continuity
            if conversation_keywords:
                keyword_matches = sum(1 for kw in conversation_keywords if kw in chunk_text)
                context_bonus += keyword_matches * 0.05
            
            # Bonus for query type relevance
            if query.lower().startswith(('what', 'define', 'explain')):
                if any(term in chunk_text for term in ['definition', 'means', 'refers to']):
                    context_bonus += 0.1
            elif query.lower().startswith(('how', 'process', 'method')):
                if any(term in chunk_text for term in ['steps', 'process', 'method', 'approach']):
                    context_bonus += 0.1
            elif query.lower().startswith(('why', 'reason', 'cause')):
                if any(term in chunk_text for term in ['because', 'reason', 'cause', 'due to']):
                    context_bonus += 0.1
            
            # Bonus for document structure
            metadata = hit.get('metadata', {})
            if metadata.get('chunk_number', 0) == 0:  # First chunk often contains key info
                context_bonus += 0.03
        
        hit['contextual_score'] = base_score + context_bonus
    
    return sorted(hits, key=lambda x: x.get('contextual_score', x.get('score', 0)), reverse=True)

def _build_context_string(hits: List[Dict], query: str, conversation_context: str) -> str:
    """Build comprehensive context string for LLM"""
    context_parts = []
    
    # Add conversation context if relevant
    if conversation_context and len(conversation_context) > 50:
        context_parts.append(f"Recent Conversation Context:\n{conversation_context}\n")
    
    # Add retrieved documents
    for i, hit in enumerate(hits, 1):
        chunk_id = hit.get('id')
        chunk_text = session_state['document_chunks'].get(chunk_id, '')
        
        if chunk_text:
            metadata = hit.get('metadata', {})
            filename = metadata.get('filename', 'Unknown')
            chunk_num = metadata.get('chunk_index', 0)
            doc_type = metadata.get('document_type', 'unknown')
            score = hit.get('contextual_score', hit.get('score', 0))
            
            context_parts.append(
                f"Source {i} [Score: {score:.3f}] - {filename} (chunk {chunk_num}, {doc_type}):\n{chunk_text}"
            )
    
    return "\n\n".join(context_parts)

print("✅ Contextual retrieval functions ready")

✅ Contextual retrieval functions ready


In [42]:
def generate_contextual_response(query: str, context: str, conversation_context: str = "") -> str:
    """Generate contextually-aware response using GravixLayer LLM"""
    print("🤖 Generating contextual response...")
    
    try:
        client = ensure_client()
        
        # Build contextual prompt
        prompt = _build_contextual_prompt(query, context, conversation_context)
        
        response = client.chat.completions.create(
            model=CONFIG['llm_model'],
            messages=[{"role": "user", "content": prompt}],
            max_tokens=600,
            temperature=0.1  # Lower temperature for more focused responses
        )
        
        raw_response = response.choices[0].message.content
        processed_response = _post_process_response(raw_response)
        
        return processed_response
        
    except Exception as e:
        return f"❌ Error generating response: {str(e)}"

def _build_contextual_prompt(query: str, context: str, conversation_context: str) -> str:
    """Build enhanced prompt with contextual instructions"""
    
    system_instructions = """You are an intelligent assistant with advanced contextual reasoning capabilities. 

Your task is to provide comprehensive, accurate, and contextually-aware responses by:

1. **Context Integration**: Use both document context AND conversation history
2. **Conversational Flow**: Maintain continuity with previous interactions
3. **Source Accuracy**: Base answers primarily on provided context
4. **Reasoning**: Show logical connections between different pieces of information
5. **Completeness**: Provide detailed yet focused answers
6. **Limitations**: Clearly state when information is insufficient"""
    
    conversation_section = ""
    if conversation_context and len(conversation_context.strip()) > 20:
        conversation_section = f"\n\nPrevious Conversation:\n{conversation_context}"
    
    prompt = f"""{system_instructions}
{conversation_section}

Document Context:
{context}

Current Question: {query}

Please provide a comprehensive answer that:
- Directly addresses the current question
- Integrates relevant information from the documents
- Maintains consistency with previous conversation
- Shows reasoning when connecting concepts
- Acknowledges any limitations or gaps in available information

Response:"""
    
    return prompt

def _post_process_response(response: str) -> str:
    """Clean up and format the generated response"""
    # Remove empty parentheses
    response = re.sub(r"\(\s*\)", "", response)
    
    # Convert bullet points
    response = response.replace("• ", "\n- ")
    
    # Clean up multiple newlines
    response = re.sub(r"\n{3,}", "\n\n", response)
    
    # Remove trailing/leading whitespace
    response = response.strip()
    
    return response

print("✅ LLM generation functions ready")

✅ LLM generation functions ready


In [56]:
# Main RAG System Functions

def setup_contextual_rag(index_name: str = "contextual-rag") -> bool:
    """Initialize the contextual RAG system"""
    display(Markdown("### 🚀 Setting up Contextual RAG System"))
    
    # Verify API key
    if not verify_api_key():
        print("❌ Setup failed: API key verification failed")
        return False
    
    # Create or find vector index
    print(f"🔍 Setting up vector index: {index_name}")
    index_id = find_or_create_index(index_name)
    
    if not index_id:
        print("❌ Setup failed: Could not create vector index")
        return False
    
    print(f"✅ Contextual RAG system ready!")
    print(f"   Index ID: {index_id}")
    print(f"   Configuration: {CONFIG['chunk_size']} chars/chunk, {CONFIG['retrieval_k']} retrieval")
    return True

def ingest_document(file_path: str, custom_name: str = None) -> bool:
    """Ingest a document into the contextual RAG system"""
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        return False
    
    filename = custom_name or os.path.basename(file_path)
    display(Markdown(f"### 📄 Ingesting Document: `{filename}`"))
    
    try:
        # Read file content
        with open(file_path, 'rb') as f:
            file_content = f.read()
        
        # Extract text
        print("🔍 Extracting text...")
        text = extract_text_from_file(file_content, filename)
        
        if not text or len(text.strip()) < 50:
            print("❌ Insufficient text extracted from document")
            return False
        
        print(f"✅ Extracted {len(text)} characters")
        
        # Create contextual chunks
        print("🧩 Creating contextual chunks...")
        chunks = chunk_text_contextually(text, filename)
        
        if not chunks:
            print("❌ No valid chunks created")
            return False
        
        print(f"✅ Created {len(chunks)} chunks")
        
        # Extract text and metadata for vector storage
        chunk_texts = [chunk['text'] for chunk in chunks]
        chunk_metadata = [chunk['metadata'] for chunk in chunks]
        
        # Upload to vector database
        print("💾 Uploading to vector database...")
        success = upsert_vectors(chunk_texts, chunk_metadata)
        
        if success:
            session_state['processed_files'].append(filename)
            print(f"✅ Document '{filename}' successfully ingested!")
            print(f"📊 Total documents: {len(session_state['processed_files'])}")
            print(f"📊 Total chunks stored: {session_state['total_chunks_stored']}")
            return True
        else:
            print("❌ Failed to upload to vector database")
            return False
            
    except Exception as e:
        print(f"❌ Error ingesting document: {str(e)}")
        return False

def query_contextual_rag(question: str, show_context: bool = True, 
                        show_retrieval_info: bool = True) -> Dict[str, Any]:
    """Query the contextual RAG system"""
    display(Markdown(f"### 🧠 Contextual Query: {question}"))
    print("═" * 80)
    
    # Get conversation context
    conversation_context = get_conversation_context()
    
    if conversation_context and show_retrieval_info:
        print("💭 Using conversation history for enhanced context...")
    
    # Perform contextual search
    print("🔍 Performing contextual retrieval...")
    hits, context = contextual_search_and_rerank(question, conversation_context)
    
    if not hits:
        print("❌ No relevant documents found")
        add_to_conversation("user", question)
        response = "I couldn't find any relevant information to answer your question. Please try rephrasing or check if documents have been uploaded."
        add_to_conversation("assistant", response)
        return {"response": response, "context": "", "hits": []}
    
    if show_context:
        display(Markdown("#### 📄 Retrieved Context:"))
        print(context[:1000] + "..." if len(context) > 1000 else context)
        print("─" * 80)
    
    # Generate contextual response
    print("🤖 Generating contextual response...")
    response = generate_contextual_response(question, context, conversation_context)
    
    # Add to conversation history
    add_to_conversation("user", question, {"retrieved_chunks": len(hits)})
    add_to_conversation("assistant", response, {
        "context_used": bool(conversation_context),
        "sources_used": len(hits)
    })
    
    # Display response
    display(Markdown("### 💡 Contextual Response:"))
    print(response)
    
    if show_retrieval_info:
        print(f"\n🧠 Retrieval Summary:")
        print(f"  • Analyzed {len(hits)} relevant chunks")
        print(f"  • Used conversation history: {'Yes' if conversation_context else 'No'}")
        print(f"  • Sources: {', '.join(set([hit.get('metadata', {}).get('filename', 'Unknown') for hit in hits]))}")
    
    print("═" * 80)
    
    return {
        "query": question,
        "response": response,
        "context": context,
        "hits": hits,
        "conversation_context_used": bool(conversation_context)
    }

def show_system_status() -> None:
    """Display current system status"""
    init_session_state()
    
    display(Markdown("### 📊 System Status"))
    
    print(f"API Key: {'✅ Set' if session_state.get('api_key_submitted') else '❌ Not set'}")
    print(f"Vector Index: {'✅ ' + session_state.get('index_id', 'None') if session_state.get('index_id') else '❌ Not created'}")
    print(f"Documents Processed: {len(session_state.get('processed_files', []))}")
    print(f"Total Chunks: {session_state.get('total_chunks_stored', 0)}")
    print(f"Conversation Messages: {len(session_state.get('chat_history', []))}")
    
    if session_state.get('processed_files'):
        print(f"\n📄 Processed Files:")
        for i, filename in enumerate(session_state['processed_files'], 1):
            print(f"  {i}. {filename}")

print("✅ Main RAG system functions ready")

✅ Main RAG system functions ready


## 7. Interactive Usage and Examples

In [44]:
# Initialize and setup the Contextual RAG system
print("🚀 Initializing Contextual RAG System...")

# Setup the system (this will verify API key and create vector index)
setup_success = setup_contextual_rag("contextual-rag-demo")

if setup_success:
    display(Markdown("### ✅ System Ready!"))
    print("\n🎯 Next steps:")
    print("1. Use `ingest_document('path/to/your/file.pdf')` to upload documents")
    print("2. Use `query_contextual_rag('Your question here')` to ask questions")
    print("3. Use `show_conversation_history()` to view chat history")
    print("4. Use `clear_conversation()` to start fresh")
else:
    display(Markdown("### ❌ Setup Failed"))
    print("Please check your API key and try again.")
    print("Use `set_api_key('your_key_here')` if needed.")

🚀 Initializing Contextual RAG System...


### 🚀 Setting up Contextual RAG System

✅ API key set successfully
✅ API key verified successfully!
🔍 Setting up vector index: contextual-rag-demo
✅ Created vector index: 6cc50e37-a768-46cb-9c6d-34d9a0c7d9fa
✅ Contextual RAG system ready!
   Index ID: 6cc50e37-a768-46cb-9c6d-34d9a0c7d9fa
   Configuration: 800 chars/chunk, 5 retrieval


### ✅ System Ready!


🎯 Next steps:
1. Use `ingest_document('path/to/your/file.pdf')` to upload documents
2. Use `query_contextual_rag('Your question here')` to ask questions
3. Use `show_conversation_history()` to view chat history
4. Use `clear_conversation()` to start fresh


In [45]:
# Example: Ingest a document (replace with your file path)

# Example 1: Ingest a PDF document
success = ingest_document('/Users/rupinajay/Downloads/Test.pdf')
if success:
    print("Document successfully uploaded!")

# Example 2: Ingest multiple documents
# documents = ['doc1.pdf', 'doc2.txt', 'doc3.docx']
# for doc_path in documents:
#     if os.path.exists(doc_path):
#         ingest_document(doc_path)

# For now, let's show the current system status
show_system_status()

### 📄 Ingesting Document: `Test.pdf`

🔍 Extracting text...
✅ Extracted 100034 characters
🧩 Creating contextual chunks...
✅ Created 167 chunks
💾 Uploading to vector database...
✅ Uploaded 167 chunks successfully
✅ Document 'Test.pdf' successfully ingested!
📊 Total documents: 1
📊 Total chunks stored: 167
Document successfully uploaded!


### 📊 System Status

🔑 API Key: ✅ Set
📇 Vector Index: ✅ 6cc50e37-a768-46cb-9c6d-34d9a0c7d9fa
📚 Documents Processed: 1
🧩 Total Chunks: 167
💬 Conversation Messages: 0

📄 Processed Files:
  1. Test.pdf


In [47]:
# Interactive Query Interface
# Replace this with your actual questions after ingesting documents

# Example queries (uncomment after uploading documents):
result = query_contextual_rag("What is the main topic of the document?")
# result = query_contextual_rag("Can you summarize the key points?")
# result = query_contextual_rag("How does this relate to what we discussed earlier?")

print("\n🔧 Utility Functions:")
print("- show_conversation_history()  # View chat history")
print("- clear_conversation()         # Clear chat history")
print("- show_system_status()         # View system status")

### 🧠 Contextual Query: What is the main topic of the document?

════════════════════════════════════════════════════════════════════════════════
💭 Using conversation history for enhanced context...
🔍 Performing contextual retrieval...
🔍 Starting contextual search for: 'What is the main topic of the document?...'
📝 Generated 4 search variations
🔍 Found 10 relevant chunks
🔍 Found 10 relevant chunks
🔍 Found 10 relevant chunks
🔍 Found 10 relevant chunks
🎯 Found 17 unique results
⭐ Selected top 3 most relevant chunks


#### 📄 Retrieved Context:

Recent Conversation Context:
Human: What is the main topic of the document?
Assistant: Based on the provided document context and conversation history, I can infer that the main topic of the document is Artificial Intelligence (AI) in the context of travel and tourism.

The introduction...


Source 1 [Score: 0.821] - Test.pdf (chunk 164, pdf):
[PAGE 44] INTRODUCTION TO AI © World Travel & Tourism Council: Introduction to AI 2024. All rights reserved. The copyright laws of the United Kingdom allow certain uses of this content without our (i.e. the copyright owner’s) permission. You are permitted to use limited extracts of this content, provided such use is fair and when such 
use is for non-commercial research, private study, review or news reporting. The following acknowledgment must also be used, whenever our content is used relying on this “fair dealing” exception: “Source: World Travel and 
Tourism Council: Introduction to AI 2024.

Source 2 [Score: 0.797] - Test.pdf (chunk 165, pdf

### 💡 Contextual Response:

Based on the provided document context, recent conversation history, and the sources listed above, I can infer that the main topic of the document is Artificial Intelligence (AI) in the context of travel and tourism.

The introduction to AI 2024, as mentioned in Source 1 [Score: 0.821], explicitly states that the copyright laws of the United Kingdom allow certain uses of this content without permission, provided such use falls under the "fair dealing" exception or complies with the Attribution, Non-Commercial 4.0 International Creative Commons Licence (Source 2 [Score: 0.797] and Source 3 [Score: 0.770]). This suggests that the document is focused on AI in travel and tourism, as it provides information about using AI in this context while adhering to copyright laws.

The consistency of the sources across different chunks of the PDF (164-166) further supports this inference. The repeated mention of "Introduction to AI 2024" and the World Travel & Tourism Council as the source indicates 

In [52]:
# Conversation Management Tools

def interactive_chat():
    """Start an interactive chat session"""
    print("🤖 Starting Interactive Contextual RAG Chat")
    print("Type 'quit' to exit, 'clear' to clear history, 'status' for system info")
    print("─" * 60)
    
    while True:
        try:
            user_input = input("\n🧑 You: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("👋 Goodbye!")
                break
            elif user_input.lower() == 'clear':
                clear_conversation()
                continue
            elif user_input.lower() == 'status':
                show_system_status()
                continue
            elif user_input.lower() == 'history':
                show_conversation_history()
                continue
            elif not user_input:
                print("Please enter a question or command.")
                continue
            
            # Process the query
            result = query_contextual_rag(user_input, show_context=False, show_retrieval_info=False)
            print(f"\n🤖 Assistant: {result['response']}")
            
        except KeyboardInterrupt:
            print("\n👋 Chat session ended.")
            break
        except Exception as e:
            print(f"❌ Error: {e}")
            continue

interactive_chat()

print("💬 Interactive chat ready! Use `interactive_chat()` to start chatting.")
print("📝 Or use individual functions for more control:")
print("   - query_contextual_rag('Your question')")
print("   - show_conversation_history()")
print("   - clear_conversation()")

🤖 Starting Interactive Contextual RAG Chat
Type 'quit' to exit, 'clear' to clear history, 'status' for system info
────────────────────────────────────────────────────────────


### 🧠 Contextual Query: Explain what all ai concepts used in this doc

════════════════════════════════════════════════════════════════════════════════
🔍 Performing contextual retrieval...
🔍 Starting contextual search for: 'Explain what all ai concepts used in this doc...'
📝 Generated 3 search variations
🔍 Found 10 relevant chunks
🔍 Found 10 relevant chunks
🔍 Found 10 relevant chunks
🎯 Found 25 unique results
⭐ Selected top 3 most relevant chunks
🤖 Generating contextual response...
🤖 Generating contextual response...


### 💡 Contextual Response:

Based on the provided document context, recent conversation history, and sources listed above, I can infer that several AI concepts are used in this document. To explain what all AI concepts used in this doc, let's break down the relevant information from the sources:

1. **Predictive AI**: According to Source 1 [Score: 0.969], Predictive AI is a type of AI system that uses current and historical data to make predictions about future events, outcomes, or behaviors. This concept is mentioned in the context of analyzing large amounts of data to identify patterns, trends, and relationships.
2. **Expert Systems**: Source 1 [Score: 0.969] also mentions Expert Systems as a way for AI systems to draw from the expertise of many stakeholders who have fed into the 'knowledge base'. This concept is relevant in the context of providing highly tailored recommendations based on individual preferences and requirements.
3. **AI Governance, Safety, and Regulation**: Source 3 [Score: 0.894] mentions tha

## 8. Advanced Features and Utilities

In [49]:
# Advanced Features

def bulk_ingest_directory(directory_path: str, supported_extensions: List[str] = None) -> List[str]:
    """Ingest all supported documents from a directory"""
    if supported_extensions is None:
        supported_extensions = ['.pdf', '.txt', '.md', '.docx']
    
    if not os.path.isdir(directory_path):
        print(f"❌ Directory not found: {directory_path}")
        return []
    
    display(Markdown(f"### 📁 Bulk Ingestion from: `{directory_path}`"))
    
    ingested_files = []
    
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        file_ext = os.path.splitext(filename)[1].lower()
        
        if os.path.isfile(file_path) and file_ext in supported_extensions:
            print(f"\n📄 Processing: {filename}")
            success = ingest_document(file_path)
            if success:
                ingested_files.append(filename)
            else:
                print(f"⚠️  Failed to ingest: {filename}")
    
    print(f"\n✅ Bulk ingestion complete: {len(ingested_files)}/{len([f for f in os.listdir(directory_path) if os.path.splitext(f)[1].lower() in supported_extensions])} files")
    return ingested_files

def search_by_metadata(metadata_filter: Dict, top_k: int = 10) -> List[Dict]:
    """Search documents by metadata filters"""
    print(f"🔍 Searching by metadata: {metadata_filter}")
    
    # Use a generic query for metadata-based search
    hits = search_vectors("*", top_k=top_k, filters=metadata_filter)
    
    if hits:
        print(f"✅ Found {len(hits)} documents matching filters")
        for i, hit in enumerate(hits, 1):
            metadata = hit.get('metadata', {})
            print(f"  {i}. {metadata.get('filename', 'Unknown')} (chunk {metadata.get('chunk_index', 0)})")
    else:
        print("❌ No documents found matching the filters")
    
    return hits

def export_conversation(format_type: str = 'json', file_path: str = None) -> str:
    """Export conversation history in various formats"""
    init_session_state()
    
    if not session_state['chat_history']:
        print("❌ No conversation history to export")
        return ""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    if format_type.lower() == 'json':
        export_data = {
            "export_timestamp": timestamp,
            "total_messages": len(session_state['chat_history']),
            "conversation": list(session_state['chat_history'])
        }
        export_content = json.dumps(export_data, indent=2, default=str)
        default_filename = f"conversation_{timestamp}.json"
    
    elif format_type.lower() == 'txt':
        lines = [f"Conversation Export - {timestamp}\n", "=" * 50 + "\n\n"]
        
        for msg in session_state['chat_history']:
            role = "You" if msg['role'] == 'user' else "Assistant"
            timestamp = datetime.fromisoformat(msg['timestamp']).strftime("%Y-%m-%d %H:%M:%S")
            lines.append(f"[{timestamp}] {role}:\n{msg['content']}\n\n")
        
        export_content = "".join(lines)
        default_filename = f"conversation_{timestamp}.txt"
    
    else:
        print(f"❌ Unsupported format: {format_type}")
        return ""
    
    output_path = file_path or default_filename
    
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(export_content)
        print(f"✅ Conversation exported to: {output_path}")
        return output_path
    except Exception as e:
        print(f"❌ Export failed: {e}")
        return ""

def get_document_summary() -> Dict[str, Any]:
    """Get summary of ingested documents"""
    init_session_state()
    
    summary = {
        "total_documents": len(session_state.get('processed_files', [])),
        "total_chunks": session_state.get('total_chunks_stored', 0),
        "document_types": {},
        "files": session_state.get('processed_files', [])
    }
    
    # Count document types
    for filename in session_state.get('processed_files', []):
        ext = os.path.splitext(filename)[1].lower()
        doc_type = _get_document_type(filename)
        summary["document_types"][doc_type] = summary["document_types"].get(doc_type, 0) + 1
    
    return summary

print("✅ Advanced features ready")
print("🔧 Available functions:")
print("   - bulk_ingest_directory('/path/to/docs')")
print("   - search_by_metadata({'document_type': 'pdf'})")
print("   - export_conversation('json')")
print("   - get_document_summary()")

✅ Advanced features ready
🔧 Available functions:
   - bulk_ingest_directory('/path/to/docs')
   - search_by_metadata({'document_type': 'pdf'})
   - export_conversation('json')
   - get_document_summary()


## 9. Final Summary and Usage Guide

In [57]:
# Complete Usage Guide
display(Markdown("# Contextual RAG Usage Guide"))

print("""
                       CONTEXTUAL RAG SYSTEM                   
                     Ready for Interactive Use                 


 GETTING STARTED:
   1  setup_contextual_rag()                    # Initialize system
   2  ingest_document('path/to/file.pdf')       # Upload documents  
   3  query_contextual_rag('Your question')     # Ask questions

 DOCUMENT MANAGEMENT:
    ingest_document('file.pdf')                # Single file
    bulk_ingest_directory('/docs')             # Entire directory
    get_document_summary()                     # Document overview
    search_by_metadata({'type': 'pdf'})       # Filter by metadata

 CONVERSATION FEATURES:
      query_contextual_rag('question')           # Context-aware queries
      interactive_chat()                         # Start chat session
      show_conversation_history()                # View chat history
      clear_conversation()                       # Clear history
      export_conversation('json')                # Export chat

 SYSTEM UTILITIES:
    show_system_status()                       # System overview
    verify_api_key()                          # Check API key
    set_api_key('your_key')                   # Set new API key

 KEY FEATURES:
    Contextual Understanding    - Maintains conversation flow
    Advanced Retrieval         - Multi-stage semantic search
    Multi-Document Support     - PDF, DOCX, TXT, Markdown
    Memory Management          - Persistent conversation history
    Query Enhancement          - Context-aware query expansion
    Rich Metadata             - Document structure preservation

""")

# Show current system status
show_system_status()

print("\n Contextual RAG system is ready for use!")
print(" Start by uploading documents, then ask questions to see the contextual magic! ")

# Contextual RAG Usage Guide


                       CONTEXTUAL RAG SYSTEM                   
                     Ready for Interactive Use                 


 GETTING STARTED:
   1  setup_contextual_rag()                    # Initialize system
   2  ingest_document('path/to/file.pdf')       # Upload documents  
   3  query_contextual_rag('Your question')     # Ask questions

 DOCUMENT MANAGEMENT:
    ingest_document('file.pdf')                # Single file
    bulk_ingest_directory('/docs')             # Entire directory
    get_document_summary()                     # Document overview
    search_by_metadata({'type': 'pdf'})       # Filter by metadata

 CONVERSATION FEATURES:
      query_contextual_rag('question')           # Context-aware queries
      interactive_chat()                         # Start chat session
      show_conversation_history()                # View chat history
      clear_conversation()                       # Clear history
      export_conversation('json')                # Export chat



### 📊 System Status

API Key: ✅ Set
Vector Index: ✅ 6cc50e37-a768-46cb-9c6d-34d9a0c7d9fa
Documents Processed: 1
Total Chunks: 167
Conversation Messages: 10

📄 Processed Files:
  1. Test.pdf

 Contextual RAG system is ready for use!
 Start by uploading documents, then ask questions to see the contextual magic! 
